In [1]:
# Imports
import numpy as np
import cv2
import matplotlib.pyplot as plt

from tensorflow import keras

In [2]:
MODEL_PATH = 'mnist_model.h5'   # Make sure this file exists (exported from digit.ipynb)

# Load model
model = keras.models.load_model(MODEL_PATH)
print("Model loaded:", MODEL_PATH)


Model loaded: mnist_model.h5


In [3]:
import time

cap = cv2.VideoCapture(0)

last_time = time.time()
digit = "?"

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    h, w = gray.shape
    x1, y1 = w//2 - 100, h//2 - 100
    x2, y2 = w//2 + 100, h//2 + 100
    roi = gray[y1:y2, x1:x2]

    roi = cv2.resize(roi, (28, 28))

    # KEY FIX
    _, roi = cv2.threshold(roi, 150, 255, cv2.THRESH_BINARY_INV)

    roi = roi / 255.0
    roi = roi.reshape(1, 784)

    # Update prediction every second
    if time.time() - last_time > 1:
        prediction = model.predict(roi, verbose=0)
        digit = np.argmax(prediction)
        last_time = time.time()

    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(frame, f"Digit: {digit}", (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    cv2.imshow("Digit Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()
